In [1]:
import numpy as np
from tifffile import imread 
import sys
import os
import cv2
from glob import glob
from tqdm import tqdm
import pandas as pd
sys.path.append('../')
from NEATUtils import MovieCreator
from NEATUtils.helpers import save_json, load_json
from NEATModels.TrainConfig import TrainConfig
from pathlib import Path


Using TensorFlow backend.


In [2]:
#Specify the directory containing image and segmentation image used for making the csv file
SourceDir = '/home/kapoorlab/CSVforNeat/NEATcsvfiles/'
#Specify the model directory where we store the json of categories, training model and parameters
ModelDir = '/home/kapoorlab/NEATModel/'
#Directory for storing center ONEAT training data for action events
CenterTrainDataDir = '/home/kapoorlab/CenterTrainData/'
#Directory for storing center ONEAT training data for static classes
StaticCenterTrainDataDir = '/home/kapoorlab/MicroscopeTrainData/'

#Raw Movie name corresponding to the csv file
MovieName = 'EventMovie'
#Segmentation Movie name corresponding to the csv file
SegMovieName = 'SegEventMovie'

#Names of the csv file begin as MovieName + EventName + csv, if any other please change the lines below
NormalCSV = SourceDir + MovieName +'Normal.csv'
ApoptosisCSV = SourceDir + MovieName +'Apoptosis.csv'
DivisionCSV = SourceDir + MovieName +'Division.csv'
MacrocheateCSV = SourceDir + MovieName +'MacroKitty.csv'
NonMatureCSV = SourceDir + MovieName +'Mature.csv'
MatureCSV = SourceDir + MovieName +'Mature.csv'



In [3]:
#Reading images and creating directories
SourceImage = imread(SourceDir + MovieName + '.tif')
SegmentationImage = imread(SourceDir + SegMovieName + '.tif')
Path(CenterTrainDataDir).mkdir(exist_ok = True)
Path(StaticCenterTrainDataDir).mkdir(exist_ok = True)
Path(ModelDir).mkdir(exist_ok = True)

In [7]:
#X Y Tminus and Tplus for making image volumes for dynamic and static events
crop_sizeM4 = [124,124,4,0]
crop_sizeM3 = [124,124,3,0]
SizeX = crop_sizeM4[0]
SizeY = crop_sizeM4[1]

#We only support training grids of 1 by 1 for yolo in time 
gridX = 1
gridY = 1
#Name of the action events
ActionName = ["Normal", "Apoptosis", "Division"]
#Vectors attached to each action event
ActionCoordinateName = ["X", "Y", "T", "H", "W", "Conf", "Angle", "SizeX", "SizeY", "SizeTminus", "SizeTplus"]
ActionCoordinateLabel = [0, 1, 2, 3, 4, 5, 6, SizeX, SizeY, crop_sizeM4[2] , crop_sizeM4[3]]
#Label corresponding to the action event
ActionEventTrain = [0, 1, 2]


In [9]:
ActionEventCSV = [NormalCSV, ApoptosisCSV, DivisionCSV]

TotalActionCategories = len(ActionEventCSV)
print('Categories for action training', TotalActionCategories)

ActionConfig = TrainConfig(ActionName, ActionEventTrain, ActionCoordinateName, ActionCoordinateLabel)

Action_json, Action_Cord_json = ActionConfig.to_json()
save_json(Action_json, ModelDir + "ActionCategories" + '.json')
save_json(Action_Cord_json, ModelDir + "ActionCord" + '.json')


Categories for action training 3
Categories for static training 4


# For center ONEAT, event is exactly in the center for all training examples

In [ ]:
defname = "MinusFourZero"
for i in range(TotalActionCategories):
     csv_file = ActionEventCSV[i]
     trainlabel = ActionEventTrain[i]   
     MovieCreator.CreateTrainingMovies(csv_file, SourceImage, SegmentationImage, crop_sizeM4, TotalActionCategories, trainlabel, CenterTrainDataDir, gridX = gridX, gridY = gridY, defname = defname)


In [ ]:
defname = "MinusThreeZero"
for i in range(TotalActionCategories):
     csv_file = ActionEventCSV[i]
     trainlabel = ActionEventTrain[i]   
     MovieCreator.CreateTrainingMovies(csv_file, SourceImage, SegmentationImage, crop_sizeM3, TotalActionCategories, trainlabel, CenterTrainDataDir, gridX = gridX, gridY = gridY, defname = defname)
